# Lab 2

## Experiment Setup

### Import python packages

In [ ]:
# @markdown Run this cell to install dependencies. The whole process will take about 30 seconds.
%%capture

!git clone https://github.com/SafeRoboticsLab/ECE346.git
% cd /content
! git clone https://github.com/mdolab/pyspline.git 
% cd pyspline 
! cp config/defaults/config.LINUX_GFORTRAN.mk config/config.mk 
! make 
! pip install .
% cd /content/ECE346/ROS_ws/src/lab2/traj_planning_ros/src
% pwd


In [ ]:
# @markdown load dependency
import os
import time
import numpy as np
import pickle
import csv
from matplotlib import pyplot as plt
from matplotlib import cm
from IPython.display import Image
import imageio
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']

from iLQR import Track, iLQR, Dynamics, plot_ellipsoids, Ellipsoid, EllipsoidObj
from iLQR import get_agent_FRS, plot_footprint, state2ell

### Set parameters for the planner

In [ ]:
#@title { run: "auto" , form-width: "30%"}

#@markdown weights in cost

w_vel =  6#@param {type:"number"}
w_contour =  30#@param {type:"number"}
w_theta =  2#@param {type:"number"}
w_accel =  0.5#@param {type:"number"}
w_delta =  0.1#@param {type:"number"}


#@markdown weights in soft constraints

# parameter for soft constraints
q1_v =  0#@param {type:"number"}
q2_v =  0#@param {type:"number"}

q1_road = 5#@param {type:"number"}
q2_road = 10#@param {type:"number"}

q1_obstacle = 3#@param {type:"number"}
q2_obstacle = 6#@param {type:"number"}

q1_lat =  1#@param {type:"number"}
q2_lat =  5#@param {type:"number"}

#@markdown iLQR Parameters
T = 1 #@param {type:"number"}
N = 11 #@param {type:"integer"}
max_itr = 50  #@param {type:"integer"}

# @markdown Uncertainty in planning
sigma_x = 0  # @param {type:"slider", min:0, max:0.1, step:0.01}
sigma_y = 0  # @param {type:"slider", min:0, max:0.1, step:0.01}
sigma_v = 0  # @param {type:"slider", min:0, max:0.1, step:0.01}
sigma_theta = 0  # @param {type:"slider", min:0, max:0.1, step:0.01}
sigma = np.array([sigma_x, sigma_y, sigma_v, sigma_theta])

params = {
    'wheelbase': 0.257,
    'width': 0.2,
    'length': 0.5,
    'm': 2.99,
    'track_width_R': 0.3,
    'track_width_L': 0.3,
    'track_offset': 0,
    'delta_min': -0.35,
    'delta_max': 0.35,
    'v_max': 2,
    'v_min': 0,
    'a_min': -3.5,
    'a_max': 3.5,
    'alat_max': 5,
    'w_vel': w_vel,
    'w_contour': w_contour,
    'w_theta': w_theta,
    'w_accel': w_accel,
    'w_delta': w_delta,
    'q1_v': q1_v,
    'q2_v': q2_v,
    'q1_road': q1_road,
    'q2_road': q2_road,
    'q1_lat': q1_lat,
    'q2_lat': q2_lat,
    'q1_obs': q1_obstacle,
    'q2_obs': q2_obstacle,
    'T': T,
    'N': N,  # the number of planning steps
    'max_itr': max_itr,
    'ego_sigma': np.zeros(4),  # the uncertainty in ego agent's dynamics
    'other_sigma':sigma,
    'track_file': 'outerloop_center_smooth.csv',
}

In [ ]:
#@title { run: "auto" , form-width: "30%"}
#@markdown # Set experiment tag for snapshot

experiment_tag = "D6"  # @param {type:"string"}

### Load track

In [ ]:
#@title { display-mode: "form" }
x = []
y = []
with open('iLQR/'+params['track_file'], newline='') as f:
  spamreader = csv.reader(f, delimiter=',')
  for i, row in enumerate(spamreader):
    if i > 0:
      x.append(float(row[0]))
      y.append(float(row[1]))

center_line = np.array([x, y])
track = Track(
    center_line=center_line, width_left=params['track_width_L'],
    width_right=params['track_width_R'], loop=False
)

## Task 2: Dynamic Obstacles: other agent's FRS

### Load other agent's policy, nominal trajectory and error dynamics
Open a folder `/content/data/` and upload your `xxx.p` files under this folder. Note that one is the policy without noise (used in the iLQR planning) and one is with noise (used in the rollout).

In [ ]:
# @markdown Load Other Vehicle's policy
filepath = os.path.join('/content', 'data')
os.makedirs(filepath, exist_ok=True)
filename = "task1_perfect.p"  # @param {type:"string"}
predict_pkl = os.path.join(filepath, filename)

obstacle = pickle.load(open(predict_pkl, "rb"))
obs_state = obstacle[0][:4, :, :]
obs_ctrl = obstacle[0][4:, :, :]

obs_K = obstacle[1]
obs_Fx = obstacle[2]
obs_Fu = obstacle[3]

subfolder = "task2_high"  # @param {type:"string"}
fig_folder = os.path.join("/content", "figure", subfolder)

In [ ]:
# @markdown Load Real Agent
filename = "task1_uncertainty.p" # @param {type:"string"}
pre_computed = os.path.join(filepath, filename)

pre_computed_dict = pickle.load(open(pre_computed, "rb"))
rollout_state = pre_computed_dict[0][:4, :, :]

### Initialize

In [ ]:
solver = iLQR(track, params)
pos0, psi0 = track.interp([0])  # The position and yaw on the track.
x_cur = np.array([pos0[0], pos0[1], 0, psi0[-1]])
init_control = np.zeros((2, params['N']))
t_total = 0
plot_cover = False

# The number of receding iterations, which cannot be longer than
# obs_state.shape[-1].
itr_receding = obs_state.shape[-1]
state_hist = np.zeros((4, itr_receding))
control_hist = np.zeros((2, itr_receding))
plan_hist = np.zeros((6, params['N'], itr_receding))

plt.figure(figsize=(15, 15))
fig_prog_folder = os.path.join(fig_folder, "progress")
os.makedirs(fig_prog_folder, exist_ok=True)

In [ ]:
# Constructs a static obstacle.
ego_a = params['length'] / 2.0
ego_b = params['width'] / 2.0
ego_q = np.array([0, 5.6])[:, np.newaxis] 
ego_Q = np.diag([ego_a**2, ego_b**2])
static_obs = EllipsoidObj(q=ego_q, Q=ego_Q)
static_obs_list = [static_obs for _ in range(params['N'])]

# plot initial
ell_a = params['length'] / 2.0
ell_b = params['width'] / 2.0
wheelbase = params['wheelbase']
ell_q = np.array([wheelbase / 2, 0])[:, np.newaxis]
ell_Q = np.diag([ell_a**2, ell_b**2])
agent = EllipsoidObj(q=ell_q, Q=ell_Q)

track.plot_track()
static_obs.plot(plt.gca(), color='k', plot_center=False)

# plot ego agent
ego = state2ell(x_cur, agent)
ego.plot(plt.gca(), color='b', plot_center=False)

# plot other agent
other = state2ell(obs_state[:, 0, 0], agent)
other.plot(plt.gca(), color='r', plot_center=False)

### iLQR Planning

In [ ]:
for i in range(itr_receding):
  # Gets other agent's FRS given the error system dynamics, linear policy,
  # nominal trajectory, and disturbance ball.
  frs = get_agent_FRS(
      obs_Fx[:, :, :, i],
      obs_Fu[:, :, :, i],
      obs_K[:, :, :, i],
      obs_state[:, :, i],
      dstb=Ellipsoid(q=np.zeros((4, 1)), Q=np.diag(params['other_sigma'])),
  )

  # Plans the trajectory by using iLQR.
  states, controls, t_process, status, theta, _, _, _ = (
      solver.solve(x_cur, controls=init_control, obs_list=[static_obs_list, frs])
  )

  # Executes the first control.
  x_cur = solver.dynamics.forward_step(
      x_cur, controls[:, 0], step=10, noise=params['ego_sigma']
  )[0]
  print(
      "[{}]: solver returns status {} and uses {:.3f}.".format(
          i, status, t_process
      ),
      end='\r'
  )
  t_total += t_process

  # Records planning history, states and controls.
  plan_hist[:4, :, i] = states
  plan_hist[4:, :, i] = controls
  state_hist[:, i] = states[:, 0]
  control_hist[:, i] = controls[:, 0]

  # Updates the nominal control signal for next receding horizon (The first
  # control is executed).
  init_control[:, :-1] = controls[:, 1:]

  # Plots the current progress with other agent's FRS.
  plt.clf()
  track.plot_track()
  plot_ellipsoids(
      plt.gca(), frs, arg_list=[dict(c='r', linewidth=1.)], dims=[0, 1], N=50,
      plot_center=False, use_alpha=True
  )
  plot_ellipsoids(
      plt.gca(), static_obs_list[0:1], arg_list=[dict(c='k', linewidth=1.)],
      dims=[0, 1], N=50, plot_center=False, use_alpha=True
  )
  plot_ellipsoids(
      plt.gca(), [solver.cost.soft_constraints.ego_ell[0]],
      arg_list=[dict(c='b')], dims=[0, 1], N=50, plot_center=False
  )
  if plot_cover:  # plot circles that cover the footprint.
    frs[0].plot_circ(plt.gca())
    solver.cost.soft_constraints.ego_ell[0].plot_circ(plt.gca())
  plt.plot(states[0, :], states[1, :], linewidth=2, c='b')
  sc = plt.scatter(
      state_hist[0, :i + 1], state_hist[1, :i + 1], s=24,
      c=state_hist[2, :i + 1], cmap=cm.jet, vmin=0, vmax=params['v_max'],
      edgecolor='none', marker='o'
  )
  cbar = plt.colorbar(sc)
  cbar.set_label(r"velocity [$m/s$]", size=20)
  plt.axis('equal')
  plt.savefig(os.path.join(fig_prog_folder, str(i)+".png"), dpi=150)

print("Planning uses {:.3f}.".format(t_total))
plt.show()
plt.close('All')

### Make experiment snapshot

In [ ]:
import PIL
def save_snapshot(experiment_tag, frame_index=itr_receding-1, gif_end_index=90):
  snapshot_folder = os.path.join("/content", "figure", "snapshots")
  os.makedirs(snapshot_folder, exist_ok=True)
  
  # save trajectory
  img = imageio.imread(os.path.join(fig_prog_folder, str(frame_index) + ".png"))
  imageio.imwrite(os.path.join(snapshot_folder, experiment_tag + ".png"), img[75:520, 160:620])
  
  # save gif
  gif_path = os.path.join(snapshot_folder, experiment_tag + "_plan.gif")
  gif_path_snap = os.path.join(snapshot_folder, experiment_tag + "_plan_snap.gif")
  with imageio.get_writer(gif_path, mode='I') as writer:
    with imageio.get_writer(gif_path_snap, mode='I') as writer_snap:
      for i in range(itr_receding):
        print(i, end='\r')
        filename = os.path.join(fig_prog_folder, str(i)+".png")
        image = imageio.imread(filename)[75:520, 160:620]
        writer.append_data(image)
        if i < gif_end_index:
          writer_snap.append_data(image)

  return img, gif_path_snap

img, gif_path = save_snapshot(experiment_tag, frame_index=99)
Image(open(gif_path,'rb').read(), width=300)

In [ ]:
# !zip /content/snapshots.zip /content/figure/snapshots/*

### Visualize

In [ ]:
plt.figure(figsize=(15, 15))
track.plot_track()
static_obs.plot(plt.gca(), color='k', plot_center=False)
plt.plot(state_hist[0, :], state_hist[1, :], 'b-o', linewidth=3, markersize=8)
plt.plot(
    obs_state[0, :state_hist.shape[1]], obs_state[1, :state_hist.shape[1]],
    'r--o', linewidth=3, markersize=8
)
plot_footprint(plt.gca(), state_hist[:, :-1], color="b")

plt.title("Trajectory")
plt.savefig(os.path.join(fig_folder, "traj.png"), dpi=200)

# velocity
plt.figure(figsize=(15, 15))
track.plot_track()
static_obs.plot(plt.gca(), color='k', plot_center=False)
sc = plt.scatter(
    state_hist[0, :], state_hist[1, :], s=80, c=state_hist[2, :], cmap=cm.jet,
    vmin=params['v_min'], vmax=params['v_max'], edgecolor='none', marker='o'
)
cbar = plt.colorbar(sc)
cbar.set_label(r"Velocity [$m/s$]", size=20)
plt.savefig(os.path.join(fig_folder, "vel.png"), dpi=200)

# Longitudinal Accel
plt.figure(figsize=(15, 15))
track.plot_track()
static_obs.plot(plt.gca(), color='k', plot_center=False)
sc = plt.scatter(
    state_hist[0, :], state_hist[1, :], s=80, c=control_hist[0, :],
    cmap=cm.jet, vmin=params['a_min'], vmax=params['a_max'], edgecolor='none',
    marker='o'
)
cbar = plt.colorbar(sc)
cbar.set_label(r"Longitudinal Accel [$m/s^2$]", size=20)
plt.savefig(os.path.join(fig_folder, "long_acc.png"), dpi=200)

# Longitudinal Accel
plt.figure(figsize=(15, 15))
track.plot_track()
static_obs.plot(plt.gca(), color='k', plot_center=False)
a_lat = np.abs(state_hist[2, :]**2 * np.tan(control_hist[1, :]) / 0.257)
# print(a_lat)
sc = plt.scatter(
    state_hist[0, :], state_hist[1, :], s=80, c=a_lat, cmap=cm.jet, vmin=0,
    vmax=params['alat_max'], edgecolor='none', marker='o'
)
cbar = plt.colorbar(sc)
cbar.set_label(r"Lateral Accel [$m/s^2$]", size=20)
plt.savefig(os.path.join(fig_folder, "lat_acc.png"), dpi=200)
plt.close("all")

In [ ]:
fig_rollout_folder = os.path.join(fig_folder, "rollout")
os.makedirs(fig_rollout_folder, exist_ok=True)

other_dyn = Dynamics(params)
traj_length = state_hist.shape[1]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
for i in range(traj_length):
  print(i, end='\r')
  ax.clear()
  track.plot_track()
  static_obs.plot(plt.gca(), color='k', plot_center=False)
  
  # plot ego agent
  ego = state2ell(state_hist[:, i], agent)
  ego.plot(ax, color='b', plot_center=False)
  
  # plot other agent
  other = state2ell(rollout_state[:, 0, i], agent)
  other.plot(ax, color='r', plot_center=False)
  
  fig.savefig(os.path.join(fig_rollout_folder, str(i)+".png"), dpi=200)
plt.close()
print("Finish rollout")  

In [ ]:
gif_path = os.path.join(fig_folder, 'rollout.gif')
with imageio.get_writer(gif_path, mode='I') as writer:
  for i in range(traj_length):
    print(i, end='\r')
    filename = os.path.join(fig_rollout_folder, str(i)+".png")
    image = imageio.imread(filename)
    writer.append_data(image)
Image(open(gif_path,'rb').read(), width=400)

In [ ]:
gif_path = os.path.join(fig_folder, 'plan.gif')
with imageio.get_writer(gif_path, mode='I') as writer:
  for i in range(itr_receding):
    print(i, end='\r')
    filename = os.path.join(fig_prog_folder, str(i)+".png")
    image = imageio.imread(filename)
    writer.append_data(image)
Image(open(gif_path,'rb').read(), width=400)

## Task 2: Simulation Results

The goal of tuning parameters is to shape the cost landscape on the track, such that iLQR can efficiently find an trajectory which is also reasonable in practice. Hence, good performance indicators for our parameters are
- **Planning time**: shorter planning time corresponds to less iLQR iterations indicating a more well-behaved cost landscape
- **Trajectory**: better trajectories corresponds to better match between cost landscape and desired behavior on the track

To describe the performance of our trajectory, we use the following terms to refer to particular stages of overtaking another truck:
- **Preparation (prep)**: The process of our truck changing to the second lane. 
- **Initiation (init)**: The process of our truck approaching the other truck on the second lane.
- **Overtaking**: The process of our truck bypassing the other truck.

### A. Tuning of iLQR parameters
*Note: Since the nominal trajectory of the other truck (task 1, experiment D6) only consists of `N = 11` planned states with a time horizon of `T = 1`, we have to use the same iLQR parameters for the planning of our truck.*

### B. Tuning of weights in cost
\#| \| |w_vel|w_countour|w_theta|w_accel|w_delta| \| |q_v|q_road|q_obstacle|q_lat| \| |sig_x|sig_y|sig_v|sig_theta| \| |T|N|max_itr| \| |comp_time|remark
-| - |-|-|-|-|-| - |-|-|-|-| - |-|-|-|-| - |-|-|-| - |-|-
base| \| |4|40|0|0.1|0.1| \| |0 / 0|5 / 10|5 / 5|1 / 5| \| |0|0|0|0| \| |1|11|50| \| |13.828|*initial configuration (prep stable, init less stable, overtakes with 1.9)*
B1| \| |**8**|40|0|0.1|0.1| \| |0 / 0|5 / 10|5 / 5|1 / 5| \| |0|0|0|0| \| |1|11|50| \| |10.876|*prep instable, init stable, overtakes with 1.8*
B2a| \| |4|**80**|0|0.1|0.1| \| |0 / 0|5 / 10|5 / 5|1 / 5| \| |0|0|0|0| \| |1|11|50| \| |18.030|*not able to overtake*
B2b*| \| |4|**20**|0|0.1|0.1| \| |0 / 0|5 / 10|5 / 5|1 / 5| \| |0|0|0|0| \| |1|11|50| \| |10.513|*prep stable, init stable, overtakes with 1.8*
B3| \| |4|40|**4**|0.1|0.1| \| |0 / 0|5 / 10|5 / 5|1 / 5| \| |0|0|0|0| \| |1|11|50| \| |15.718|*prep less stable, init stable, overtakes with 1.9, hits the other truck shortly, cuts the curves heavily*
B4| \| |4|40|0|**0.5**|0.1| \| |0 / 0|5 / 10|5 / 5|1 / 5| \| |0|0|0|0| \| |1|11|50| \| |9.970|*prep almost stable, init less stable, overtakes with 1.5, requires more time to overtake, less deceleration*
B5| \| |4|40|0|0.1|**0.5**| \| |0 / 0|5 / 10|5 / 5|1 / 5| \| |0|0|0|0| \| |1|11|50| \| |10.934|*prep almost stable, init less stable, overtakes with 1.8, requires more time than base to plan maneuver*
B6| \| |**6**|40|**2**|0.1|0.1| \| |0 / 0|5 / 10|5 / 5|1 / 5| \| |0|0|0|0| \| |1|11|50| \| |15.666|*prep less stable, init stable, overtakes with 1.8*
B6b| \| |**6**|**20**|**2**|0.1|0.1| \| |0 / 0|5 / 10|5 / 5|1 / 5| \| |0|0|0|0| \| |1|11|50| \| |16.724|*?*
B7*| \| |**6**|40|**2**|**0.5**|0.1| \| |0 / 0|5 / 10|5 / 5|1 / 5| \| |0|0|0|0| \| |1|11|50| \| |15.334|*prep stable, init stable, overtakes with 1.9*
B7b| \| |**6**|**20**|**2**|**0.5**|0.1| \| |0 / 0|5 / 10|5 / 5|1 / 5| \| |0|0|0|0| \| |1|11|50| \| |19.273|*?*

### C. Tuning of weights in soft constraint
\#| \| |w_vel|w_countour|w_theta|w_accel|w_delta| \| |q_v|q_road|q_obstacle|q_lat| \| |sig_x|sig_y|sig_v|sig_theta| \| |T|N|max_itr| \| |comp_time|remark
-| - |-|-|-|-|-| - |-|-|-|-| - |-|-|-|-| - |-|-|-| - |-|-
base| \| |4|40|0|0.1|0.1| \| |0 / 0|5 / 10|5 / 5|1 / 5| \| |0|0|0|0| \| |1|11|50| \| |13.828|*initial configuration (prep stable, init less stable, overtakes with 1.9)*
C1| \| |4|40|0|0.1|0.1| \| |0 / 0|**3 / 10**|5 / 5|1 / 5| \| |0|0|0|0| \| |1|11|50| \| |9.121|*prep stable, init less stable, overtakes with 1.9, bumps into track border few times*
C2| \| |4|40|0|0.1|0.1| \| |0 / 0|**3 / 5**|5 / 5|1 / 5| \| |0|0|0|0| \| |1|11|50| \| |9.583|*prep less stable, init less stable, overtakes with 1.9, bumps into track border more heavily times*
C3*| \| |4|40|0|0.1|0.1| \| |0 / 0|5 / 10|**3 / 6**|1 / 5| \| |0|0|0|0| \| |1|11|50| \| |9.586|*prep stable, init stable, overtakes with 1.8, merges back earlier*
C4| \| |4|40|0|0.1|0.1| \| |0 / 0|5 / 10|**5 / 10**|1 / 5| \| |0|0|0|0| \| |1|11|50| \| |11.291|*prep stable, init less stable, overtakes with 1.8*
C5| \| |4|40|0|0.1|0.1| \| |0 / 0|**5 / 5**|**5 / 5**|1 / 5| \| |0|0|0|0| \| |1|11|50| \| |10.296|*prep instable, init instable, overtakes with 1.8, drives close to track border*

### D. Joint tuning
\#| \| |w_vel|w_countour|w_theta|w_accel|w_delta| \| |q_v|q_road|q_obstacle|q_lat| \| |sig_x|sig_y|sig_v|sig_theta| \| |T|N|max_itr| \| |comp_time|remark
-| - |-|-|-|-|-| - |-|-|-|-| - |-|-|-|-| - |-|-|-| - |-|-
base| \| |4|40|0|0.1|0.1| \| |0 / 0|5 / 10|5 / 5|1 / 5| \| |0|0|0|0| \| |1|11|50| \| |13.828|*initial configuration (prep stable, init less stable, overtakes with 1.9)*
D1| \| |**8**|40|0|0.1|0.1| \| |0 / 0|5 / 10|**3 / 6**|1 / 5| \| |0|0|0|0| \| |1|11|50| \| |11.291|*?*
D2| \| |4|40|**4**|0.1|0.1| \| |0 / 0|5 / 10|**3 / 6**|1 / 5| \| |0|0|0|0| \| |1|11|50| \| |13.634|*?*
D3| \| |**6**|40|**2**|0.1|0.1| \| |0 / 0|5 / 10|**3 / 6**|1 / 5| \| |0|0|0|0| \| |1|11|50| \| |16.477|*prep instable, init stable, overtakes with 1.9, merges back early*
D4| \| |**6**|40|**2**|**0.5**|0.1| \| |0 / 0|5 / 10|**3 / 6**|1 / 5| \| |0|0|0|0| \| |1|11|50| \| |14.229|*prep stable, init stable, overtakes with 1.9*
D5*| \| |**6**|**30**|**2**|**0.5**|0.1| \| |0 / 0|5 / 10|**3 / 6**|1 / 5| \| |0|0|0|0| \| |1|11|50| \| |15.254|*prep stable, init stable, overtakes with 1.9, touches other truck and track border*
D6| \| |**2**|**30**|**2**|**0.5**|0.1| \| |0 / 0|5 / 10|**3 / 6**|1 / 5| \| |0|0|0|0| \| |1|11|50| \| |16.124|*prep stable, init stable, overtakes with 1.7, does not touch other truck and track border*
D7| \| |**3**|**30**|**2**|**0.5**|0.1| \| |0 / 0|5 / 10|**3 / 6**|1 / 5| \| |0|0|0|0| \| |1|11|50| \| |15.981|*prep stable, init stable, overtakes with 1.8, touches other truck very shortly*
D8| \| |**4**|**30**|**2**|**0.5**|0.1| \| |0 / 0|5 / 10|**3 / 6**|1 / 5| \| |0|0|0|0| \| |1|11|50| \| |16.652|*prep stable, init stable, overtakes with 1.85, touches other truck and track border very shortly*
D9*| \| |**3**|**35**|**2**|**0.5**|0.1| \| |0 / 0|5 / 10|**3 / 6**|1 / 5| \| |0|0|0|0| \| |1|11|50| \| |15.748|*prep stable, init stable, overtakes with 1.75, does not touch other truck and track border*

### Discussion
- `w_vel` and `w_theta` larger --> provides incentive for the truck to overtake the other truck (by allowing higher speeds), might regularize cost landscape to more efficient trajectories
- `w_vel` and `w_theta` not too large --> prevents truck to drive too aggressively (e.g. cutting curves, overtaking too closely)
- `w_contour` small --> provides incentive for the truck to overtake the other truck and to merge back later (by allowing larger deviations from reference track)
- `w_contour` not too small --> prevents truck to cut curves too aggressively
- `w_accel` larger --> might regularize cost landscape to trajectories with less acceleration and deceleration
- *`w_delta` larger --> might regularize cost landscape to more straight trajectories with less oscillations (??)*
- `q2_road <> q2_obstacle` (i.e. different rates of exponential increase for road and obstacle constraints) --> might reduce oscillations when computing the trajectory
- One can see significant decrease in computation time when tuning the barrier function parameters. This indicates that the cost landscape is more well-behaved when using tuned barrier functions.

The oscillation during initiation of the overtaking maneuver comes from the problem that in the beginning there is no explicit incentive for iLQR to find a trajectory such that the truck overtakes the other truck from the left side. Hence, the truck oscillates between overtaking from the right or left side until it gets close enough to the other truck.